### 2021: Week 29 - PD x WOW - Tokyo 2020 Calendar

https://preppindata.blogspot.com/2021/07/2021-week-29-pd-x-wow-tokyo-2020.html

July 20, 2021
Challenge by Tom Prowse with collaboration with the Workout Wednesday team!

This week is time for our annual get together with Workout Wednesday for a joint challenge so that you can have a full data prep to visualisation solution. 

Unfortunately the Olympics was postponed in 2020, so for last year's collaboration we looked at historical winners through the history of the games. However, this year, Japan 2020 is going ahead so we thought it would be the perfect time to create an event calendar to help us keep track of the events that we don't want to miss. 

Inputs
The data comes from the Olympics website. (Note; this was taken on Wednesday 14th July so the schedule for some events may have changed since!). You can download the data here. 

### Requirements
Input the Data 

Create a correctly formatted DateTime field (hint)

Parse the event list so each event is on a separate row (hint)

Group similar sports into a Sport Type field (hint)

Combine the Venue table (hint)

Calculate whether the event is a 'Victory Ceremony' or 'Gold Medal' event. 

(Note, this might not pick up all of the medal events.)

In [1]:
import os
import pandas as pd
import shutil
import datetime as dt
envir = os.environ['USERPROFILE']

In [217]:
# #copy data
# from_file = fr'{envir}\downloads\Olympic Event Schedule.xlsx'
# to_file = fr'{envir}\Documents\02 - Weekly\07 - Preppin data\20210722\Olympic Event Schedule.xlsx'
# shutil.move(from_file,to_file)

'C:\\Users\\nprol195\\Documents\\02 - Weekly\\07 - Preppin data\\20210722\\Olympic Event Schedule.xlsx'

In [2]:
to_file = fr'{envir}\Documents\02 - Weekly\07 - Preppin data\20210722\Olympic Events.xlsx'
events = pd.read_excel(to_file,sheet_name = 'Olympics Events')
venues = pd.read_excel(to_file,sheet_name = 'Venues')

In [3]:
for a in [events,venues]:
    a.columns = [i.lower().replace(' ','_').strip() for i in a ]
events['date'].unique()   

array(['21st_July_2021', '22nd_July_2021', '22_July_2021',
       '23rd_July_2021', '24th_July_2021', '25th_July_2021',
       '26th_July_2021', '27th_July_2021', '28th_July_2021',
       '29th_July_2021', '30th_July_2021', '31st_July_2021',
       '1st_August_2021', '2nd_August_2021', '3rd_August_2021',
       '4th_August_2021', '5th_August_2021', '6th_August_2021',
       '7th_August_2021', '8th_August_2021'], dtype=object)

In [4]:
events['date']= events['date'].replace({'22_July_2021':'22nd_July_2021'})
events['day'] = events['date'].apply(lambda x:x.split('_')[0][:-2])

events['month'] = events['date'].apply(lambda x:x.split('_')[1])
events['year'] = events['date'].apply(lambda x:x.split('_')[2])
events['day']=events['day'].apply(lambda x:x.zfill(2))

events['date1'] = events['year']+events['month']+events['day']
events['date'] = pd.to_datetime(events['date1'],format='%Y%B%d')

In [5]:
events.drop(columns=['day','month','year','date1'],inplace=True)

In [6]:
eventclms = events['events'].str.split(',',expand=True)


In [7]:
eventscopy = events.merge(eventclms,left_index=True,right_index=True).copy()
eventscopy.drop(columns='events',inplace=True)
eventscopy = eventscopy.melt(id_vars=['date',  'time', 'sport', 'venue']).copy()
eventscopy= eventscopy[~eventscopy['value'].isna()].copy()
eventscopy.drop(columns='variable',inplace=True)
eventscopy.rename(columns={'value':'event'},inplace=True)
eventscopy.reset_index(drop=True,inplace=True)
eventscopy

,date,time,sport,venue,event
0,2021-07-21,1:00,Baseball/Softball,Fukushima Azuma Baseball Stadium,Australia vs Japan
1,2021-07-21,8:30,Football,Sapporo Dome,Women's Group E: Great Britain vs Chile
2,2021-07-21,9:00,Football,Miyagi Stadium,Women's Group F: China vs Brazil
3,2021-07-21,9:30,Football,Tokyo Stadium,Women's Group G: Sweden vs United States
4,2021-07-22,1:00,Baseball/Softball,Fukushima Azuma Baseball Stadium,United States vs Canada
...,...,...,...,...,...
1890,2021-07-28,0:30,Rowing,Sea Forest Waterway,Men's Single Sculls Semifinals A/B
1891,2021-07-29,0:30,Rowing,Sea Forest Waterway,Lightweight Men's Double Sculls Final B
1892,2021-07-29,0:30,Rowing,Sea Forest Waterway,Lightweight Women's Double Sculls Final B
1893,2021-07-29,0:30,Rowing,Sea Forest Waterway,Women's Single Sculls Final D


In [8]:
eventscopy['sport'] = eventscopy['sport'].str.title()

In [9]:
mapper = {'Artistic Gymnastic':'Artistic Gymnastics',
 'Baseball/Softball':'Baseball',
 'Beach Volley':'Beach Volleyball',
 'Beach Volleybal':'Beach Volleyball',
 'Beach Volleyball':'Beach Volleyball',
 'Beach volleyball':'Beach Volleyball',
 'Boxing.':'Boxing',
 'boxing':'Boxing',
 'Rugby.': 'Rugby',
 'Skateboarding.':'Skateboarding',
 'Softball': 'Softball/Baseball',
 'Wrestling.':'Wrestling',
 'rugby':'Rugby',
 'volleyball':'Volleyball'
}

In [10]:
eventscopy['sport'].replace(mapper,inplace=True)

In [11]:
eventscopy['medal_ceremony'] = eventscopy['event'].str.contains('Medal')

In [12]:
venuescopy = venues[['venue','location']].drop_duplicates()

In [13]:
eventscopy2=eventscopy.merge(venuescopy[['venue','location']],how='left',on='venue').reset_index(drop=True).copy()

In [14]:
eventscopy2


,date,time,sport,venue,event,medal_ceremony,location
0,2021-07-21,1:00,Baseball,Fukushima Azuma Baseball Stadium,Australia vs Japan,False,"37.72216480340486, 140.3640114979229"
1,2021-07-21,8:30,Football,Sapporo Dome,Women's Group E: Great Britain vs Chile,False,"43.01517544330762, 141.41041300340524"
2,2021-07-21,9:00,Football,Miyagi Stadium,Women's Group F: China vs Brazil,False,"38.33557331725407, 140.95096377309127"
3,2021-07-21,9:30,Football,Tokyo Stadium,Women's Group G: Sweden vs United States,False,"35.66446761779039, 139.52756286092847"
4,2021-07-22,1:00,Baseball,Fukushima Azuma Baseball Stadium,United States vs Canada,False,"37.72216480340486, 140.3640114979229"
...,...,...,...,...,...,...,...
1890,2021-07-28,0:30,Rowing,Sea Forest Waterway,Men's Single Sculls Semifinals A/B,False,"35.60796457817903, 139.8147625026683"
1891,2021-07-29,0:30,Rowing,Sea Forest Waterway,Lightweight Men's Double Sculls Final B,False,"35.60796457817903, 139.8147625026683"
1892,2021-07-29,0:30,Rowing,Sea Forest Waterway,Lightweight Women's Double Sculls Final B,False,"35.60796457817903, 139.8147625026683"
1893,2021-07-29,0:30,Rowing,Sea Forest Waterway,Women's Single Sculls Final D,False,"35.60796457817903, 139.8147625026683"


In [16]:
eventscopy2.to_csv('test.csv',index=False)
os.startfile('test.csv')